In [1]:
from datasets import load_dataset, Dataset
from datasets.dataset_dict import DatasetDict
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments
from encodec import EncodecModel
from encodec.utils import convert_audio
import torch
import torchaudio
import os
from tqdm import tqdm

In [2]:
dataset_train = load_dataset("mozilla-foundation/common_voice_11_0", "en", split="train[1%]")
dataset_test = load_dataset("mozilla-foundation/common_voice_11_0", "en", split="test[1%]")
dataset_val = load_dataset("mozilla-foundation/common_voice_11_0", "en", split="validation[1%]")

Found cached dataset common_voice_11_0 (C:/Users/Admin/.cache/huggingface/datasets/mozilla-foundation___common_voice_11_0/en/11.0.0/2c65b95d99ca879b1b1074ea197b65e0497848fd697fdb0582e0f6b75b6f4da0)


ValueError: Unrecognized instruction format: train[1%]

In [ ]:
datasets = DatasetDict({"train": dataset_train, "test": dataset_test, "validation": dataset_val})
datasets

DatasetDict({
    train: Dataset({
        features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment'],
        num_rows: 10
    })
    test: Dataset({
        features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment'],
        num_rows: 10
    })
    validation: Dataset({
        features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment'],
        num_rows: 10
    })
})

In [ ]:
encodec_model = EncodecModel.encodec_model_24khz()
encodec_model.set_target_bandwidth(1.5)

def tokenize_audio(audio_file):
    wav, sr = torchaudio.load(audio_file)
    wav = convert_audio(wav, sr, encodec_model.sample_rate, encodec_model.channels)
    wav = wav.unsqueeze(0)

    with torch.no_grad():
        frames = encodec_model.encode(wav)
    frame = frames[0][0][0]

    number_of_codebooks, number_of_samples = frame.shape

    tokens = []
    for sample in range(number_of_samples):
        for codebook in range(number_of_codebooks):
            token = frame[codebook, sample].tolist()
            tokens.append(token)

    return tokens

In [ ]:
model_name = "distilgpt2"
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.add_tokens([f"audio_token_{i}" for i in range(1024)])
model.resize_token_embeddings(len(tokenizer))

Embedding(51281, 768)

In [ ]:
model.config.pad_token_id = tokenizer.eos_token_id

In [ ]:
def tokenize_function(voice_recording, dataset_type="train"):
    # Get the directory of the audio file
    audio_dir = os.path.dirname(voice_recording["path"])
    audio_dir += f"/en_{dataset_type}_0/"
    # Add filename
    audio_dir += os.path.basename(voice_recording["path"])
    audio_tokens = tokenize_audio(audio_dir)

    new_tokens = [f"audio_token_{i}" for i in range(len(audio_tokens))]

    tokens = tokenizer("text: " + voice_recording["sentence"] + "\nsound: " + "".join(new_tokens))
    #tokens = tokenizer("text: " + voice_recording["sentence"] + "\nsound: ")
    if len(tokens["input_ids"]) > 1024:
        return None
    return tokens

In [ ]:
print(tokenize_function(datasets["train"][0]))
print(len(tokenize_function(datasets["train"][0])["input_ids"]))
# Sample tokenize and detokenize
tokenizer.decode(tokenize_function(datasets["train"][0])["input_ids"])

{'input_ids': [5239, 25, 383, 2610, 3568, 319, 262, 23340, 5062, 366, 42, 1617, 5225, 1911, 198, 23661, 25, 220, 50257, 50258, 50259, 50260, 50261, 50262, 50263, 50264, 50265, 50266, 50267, 50268, 50269, 50270, 50271, 50272, 50273, 50274, 50275, 50276, 50277, 50278, 50279, 50280, 50281, 50282, 50283, 50284, 50285, 50286, 50287, 50288, 50289, 50290, 50291, 50292, 50293, 50294, 50295, 50296, 50297, 50298, 50299, 50300, 50301, 50302, 50303, 50304, 50305, 50306, 50307, 50308, 50309, 50310, 50311, 50312, 50313, 50314, 50315, 50316, 50317, 50318, 50319, 50320, 50321, 50322, 50323, 50324, 50325, 50326, 50327, 50328, 50329, 50330, 50331, 50332, 50333, 50334, 50335, 50336, 50337, 50338, 50339, 50340, 50341, 50342, 50343, 50344, 50345, 50346, 50347, 50348, 50349, 50350, 50351, 50352, 50353, 50354, 50355, 50356, 50357, 50358, 50359, 50360, 50361, 50362, 50363, 50364, 50365, 50366, 50367, 50368, 50369, 50370, 50371, 50372, 50373, 50374, 50375, 50376, 50377, 50378, 50379, 50380, 50381, 50382, 50383

'text: The track appears on the compilation album "Kraftworks".\nsound: audio_token_0audio_token_1audio_token_2audio_token_3audio_token_4audio_token_5audio_token_6audio_token_7audio_token_8audio_token_9audio_token_10audio_token_11audio_token_12audio_token_13audio_token_14audio_token_15audio_token_16audio_token_17audio_token_18audio_token_19audio_token_20audio_token_21audio_token_22audio_token_23audio_token_24audio_token_25audio_token_26audio_token_27audio_token_28audio_token_29audio_token_30audio_token_31audio_token_32audio_token_33audio_token_34audio_token_35audio_token_36audio_token_37audio_token_38audio_token_39audio_token_40audio_token_41audio_token_42audio_token_43audio_token_44audio_token_45audio_token_46audio_token_47audio_token_48audio_token_49audio_token_50audio_token_51audio_token_52audio_token_53audio_token_54audio_token_55audio_token_56audio_token_57audio_token_58audio_token_59audio_token_60audio_token_61audio_token_62audio_token_63audio_token_64audio_token_65audio_token_66

In [ ]:
tokenizer.decode([50646])

'audio_token_389'

In [ ]:
datasets["train"].column_names

['client_id',
 'path',
 'audio',
 'sentence',
 'up_votes',
 'down_votes',
 'age',
 'gender',
 'accent',
 'locale',
 'segment']

In [ ]:
for recording in tqdm(datasets["train"]):
    tokenize_function(recording)

100%|██████████| 10/10 [00:04<00:00,  2.25it/s]


In [ ]:
tokenize_function(datasets["train"][0])

{'input_ids': [5239, 25, 383, 2610, 3568, 319, 262, 23340, 5062, 366, 42, 1617, 5225, 1911, 198, 23661, 25, 220, 50257, 50258, 50259, 50260, 50261, 50262, 50263, 50264, 50265, 50266, 50267, 50268, 50269, 50270, 50271, 50272, 50273, 50274, 50275, 50276, 50277, 50278, 50279, 50280, 50281, 50282, 50283, 50284, 50285, 50286, 50287, 50288, 50289, 50290, 50291, 50292, 50293, 50294, 50295, 50296, 50297, 50298, 50299, 50300, 50301, 50302, 50303, 50304, 50305, 50306, 50307, 50308, 50309, 50310, 50311, 50312, 50313, 50314, 50315, 50316, 50317, 50318, 50319, 50320, 50321, 50322, 50323, 50324, 50325, 50326, 50327, 50328, 50329, 50330, 50331, 50332, 50333, 50334, 50335, 50336, 50337, 50338, 50339, 50340, 50341, 50342, 50343, 50344, 50345, 50346, 50347, 50348, 50349, 50350, 50351, 50352, 50353, 50354, 50355, 50356, 50357, 50358, 50359, 50360, 50361, 50362, 50363, 50364, 50365, 50366, 50367, 50368, 50369, 50370, 50371, 50372, 50373, 50374, 50375, 50376, 50377, 50378, 50379, 50380, 50381, 50382, 50383

In [ ]:
def tokenize_datasets(datasets):
    def tokenize_dataset(dataset):
        # Use tokenizer function for each row in dataset, and create a new dataset
        # dont use dataset.map function
        tokenized_datasets = [tokenize_function(row, dataset if not dataset == "validation" else "dev") for row in tqdm(datasets[dataset], f"Tokenizing {dataset} dataset")]
        tokenized_datasets = [tokenized_dataset for tokenized_dataset in tokenized_datasets if tokenized_dataset is not None]
        tokens = [tokenized_dataset["input_ids"] for tokenized_dataset in tokenized_datasets]
        attention_mask = [tokenized_dataset["attention_mask"] for tokenized_dataset in tokenized_datasets]
        new_dataset = Dataset.from_dict({"input_ids": tokens, "attention_mask": attention_mask})
        return new_dataset

    new_datasets = DatasetDict(
        {"train": tokenize_dataset("train"), "test": tokenize_dataset("test"), "validation": tokenize_dataset("validation")}
    )
    return new_datasets

In [ ]:
tokenized_datasets = tokenize_datasets(datasets)

Tokenizing validation dataset: 100%|██████████| 10/10 [00:03<00:00,  2.56it/s]


In [ ]:
#tokenized_datasets = datasets.map(tokenize_function, batched=True, num_proc=1, remove_columns=datasets["train"].column_names)
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 7
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 8
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 7
    })
})

In [ ]:
block_size = 1024
def pad_samples(sample):
    # Pad each sample to the block size, with the padding token
    result = {}
    result["input_ids"] = sample["input_ids"] + [0] * (block_size - len(sample["input_ids"]))
    result["attention_mask"] = sample["attention_mask"] + [0] * (block_size - len(sample["attention_mask"]))
    result["labels"] = result["input_ids"].copy() 
    return result

In [ ]:
lm_datasets = tokenized_datasets.map(pad_samples, num_proc=1)

In [ ]:
len(lm_datasets["train"][2]["input_ids"])
print(lm_datasets["train"][2])

{'input_ids': [5239, 25, 383, 42823, 318, 10016, 351, 262, 3265, 2877, 4632, 1088, 262, 24768, 13, 198, 23661, 25, 220, 50257, 50258, 50259, 50260, 50261, 50262, 50263, 50264, 50265, 50266, 50267, 50268, 50269, 50270, 50271, 50272, 50273, 50274, 50275, 50276, 50277, 50278, 50279, 50280, 50281, 50282, 50283, 50284, 50285, 50286, 50287, 50288, 50289, 50290, 50291, 50292, 50293, 50294, 50295, 50296, 50297, 50298, 50299, 50300, 50301, 50302, 50303, 50304, 50305, 50306, 50307, 50308, 50309, 50310, 50311, 50312, 50313, 50314, 50315, 50316, 50317, 50318, 50319, 50320, 50321, 50322, 50323, 50324, 50325, 50326, 50327, 50328, 50329, 50330, 50331, 50332, 50333, 50334, 50335, 50336, 50337, 50338, 50339, 50340, 50341, 50342, 50343, 50344, 50345, 50346, 50347, 50348, 50349, 50350, 50351, 50352, 50353, 50354, 50355, 50356, 50357, 50358, 50359, 50360, 50361, 50362, 50363, 50364, 50365, 50366, 50367, 50368, 50369, 50370, 50371, 50372, 50373, 50374, 50375, 50376, 50377, 50378, 50379, 50380, 50381, 50382

In [ ]:
# convert lm_dataset to only be 128 tokens long for each row
def truncate_row(row):
    new_row = {}
    new_row["input_ids"] = row["input_ids"][:128]
    new_row["attention_mask"] = row["attention_mask"][:128]
    new_row["labels"] = row["labels"][:128]
    return new_row

#new_lm_datasets = lm_datasets.map(truncate_row, num_proc=1)
new_lm_datasets = lm_datasets

In [ ]:
new_lm_datasets.save_to_disk("CV_new_lm_datasets")

In [ ]:
print(new_lm_datasets["train"][2])

{'input_ids': [5239, 25, 383, 42823, 318, 10016, 351, 262, 3265, 2877, 4632, 1088, 262, 24768, 13, 198, 23661, 25, 220, 50257, 50258, 50259, 50260, 50261, 50262, 50263, 50264, 50265, 50266, 50267, 50268, 50269, 50270, 50271, 50272, 50273, 50274, 50275, 50276, 50277, 50278, 50279, 50280, 50281, 50282, 50283, 50284, 50285, 50286, 50287, 50288, 50289, 50290, 50291, 50292, 50293, 50294, 50295, 50296, 50297, 50298, 50299, 50300, 50301, 50302, 50303, 50304, 50305, 50306, 50307, 50308, 50309, 50310, 50311, 50312, 50313, 50314, 50315, 50316, 50317, 50318, 50319, 50320, 50321, 50322, 50323, 50324, 50325, 50326, 50327, 50328, 50329, 50330, 50331, 50332, 50333, 50334, 50335, 50336, 50337, 50338, 50339, 50340, 50341, 50342, 50343, 50344, 50345, 50346, 50347, 50348, 50349, 50350, 50351, 50352, 50353, 50354, 50355, 50356, 50357, 50358, 50359, 50360, 50361, 50362, 50363, 50364, 50365, 50366, 50367, 50368, 50369, 50370, 50371, 50372, 50373, 50374, 50375, 50376, 50377, 50378, 50379, 50380, 50381, 50382

In [ ]:
training_args = TrainingArguments(
    f"{model_name}-finetuned-common-voice",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=new_lm_datasets["train"],
    eval_dataset=new_lm_datasets["validation"],
)

In [ ]:
trainer.train()

c:\Python38\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 7
  Num Epochs = 3
  Instantaneous batch size per device = 1
  Total train batch size (w. parallel, distributed & accumulation) = 2
  Gradient Accumulation steps = 1
  Total optimization steps = 12
  Number of trainable parameters = 82699008
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: anforsm (dt2112g1). Use `wandb login --relogin` to force relogin


  0%|          | 0/12 [00:00<?, ?it/s]c:\Python38\lib\site-packages\torch\nn\parallel\_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
c:\Python38\lib\site-packages\torch\cuda\nccl.py:15: UserWarning: PyTorch is not compiled with NCCL support
  warnings.warn('PyTorch is not compiled with NCCL support')
 33%|███▎      | 4/12 [00:05<00:06,  1.30it/s]***** Running Evaluation *****
  Num examples = 7
  Batch size = 2
                                              
 33%|███▎      | 4/12 [00:05<00:06,  1.30it/s]

{'eval_loss': 21.9819393157959, 'eval_runtime': 0.3348, 'eval_samples_per_second': 20.907, 'eval_steps_per_second': 11.947, 'epoch': 1.0}


 67%|██████▋   | 8/12 [00:06<00:01,  2.92it/s]***** Running Evaluation *****
  Num examples = 7
  Batch size = 2
                                              
 67%|██████▋   | 8/12 [00:06<00:01,  2.92it/s]

{'eval_loss': 12.044716835021973, 'eval_runtime': 0.3337, 'eval_samples_per_second': 20.977, 'eval_steps_per_second': 11.987, 'epoch': 2.0}


100%|██████████| 12/12 [00:07<00:00,  3.78it/s]***** Running Evaluation *****
  Num examples = 7
  Batch size = 2
                                               
100%|██████████| 12/12 [00:07<00:00,  3.78it/s]

Training completed. Do not forget to share your model on huggingface.co/models =)


100%|██████████| 12/12 [00:07<00:00,  1.55it/s]

{'eval_loss': 8.95007610321045, 'eval_runtime': 0.3324, 'eval_samples_per_second': 21.058, 'eval_steps_per_second': 12.033, 'epoch': 3.0}
{'train_runtime': 10.6093, 'train_samples_per_second': 1.979, 'train_steps_per_second': 1.131, 'train_loss': 11.92264175415039, 'epoch': 3.0}


TrainOutput(global_step=12, training_loss=11.92264175415039, metrics={'train_runtime': 10.6093, 'train_samples_per_second': 1.979, 'train_steps_per_second': 1.131, 'train_loss': 11.92264175415039, 'epoch': 3.0})

In [ ]:

from huggingface_hub import notebook_login
notebook_login()

In [ ]:

finetuned_model_name = f"{model_name}-finetuned-common-voice-1%"
trainer.save_model(finetuned_model_name)
tokenizer.save_pretrained(finetuned_model_name)
trainer.push_to_hub(finetuned_model_name)

Saving model checkpoint to distilgpt2-finetuned-common-voice-7samples
Configuration saved in distilgpt2-finetuned-common-voice-7samples\config.json
Configuration saved in distilgpt2-finetuned-common-voice-7samples\generation_config.json
Model weights saved in distilgpt2-finetuned-common-voice-7samples\pytorch_model.bin
tokenizer config file saved in distilgpt2-finetuned-common-voice-7samples\tokenizer_config.json
Special tokens file saved in distilgpt2-finetuned-common-voice-7samples\special_tokens_map.json


OSError: Tried to clone a repository in a non-empty folder that isn't a git repository ('c:\Users\Admin\Documents\GitHub\transformer-audio\gpt-sw3\distilgpt2-finetuned-common-voice'). If you really want to do this, do it manually:
 cd c:\Users\Admin\Documents\GitHub\transformer-audio\gpt-sw3\distilgpt2-finetuned-common-voice && git init && git remote add origin && git pull origin main
 or clone repo to a new folder and move your existing files there afterwards.

In [ ]:
import math
eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

***** Running Evaluation *****
  Num examples = 7
  Batch size = 2
100%|██████████| 4/4 [00:00<00:00, 15.71it/s]

Perplexity: 7708.48


In [ ]:
input_ids = torch.tensor(tokenizer.encode("text: Hello, my name is\nsound: ", add_special_tokens=False)).unsqueeze(0).cuda()  # Batch size 1
tokens = model.generate(input_ids, max_length=1024, do_sample=True, top_k=50, top_p=0.95, temperature=0.9, num_return_sequences=1)
tokens

Generate config GenerationConfig {
  "bos_token_id": 50256,
  "eos_token_id": 50256,
  "pad_token_id": 50256,
  "transformers_version": "4.26.1"
}

c:\Python38\lib\site-packages\transformers\generation\utils.py:1186: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(


tensor([[ 5239,    25, 18435,  ..., 50436, 50920, 50256]], device='cuda:0')

In [ ]:
decoded_text = tokenizer.decode(tokens[0])
print(decoded_text)
audio_tokens = decoded_text.split("sound: ")[1].split("text: ")[0].split(" ")[0]
audio_tokens = audio_tokens.replace("audio_token_", " ")
print(audio_tokens)
# extract the integer audio tokens
audio_tokens = [int(s) for s in audio_tokens.split(" ") if s.isdigit()]
print(audio_tokens)

text: Hello, my name is
sound: audio_token_195audio_token_69audio_token_195audio_token_589audio_token_743audio_token_589audio_token_195audio_token_64audio_token_195audio_token_675audio_token_682audio_token_981audio_token_155audio_token_69audio_token_458audio_token_564audio_token_64audio_token_195audio_token_155audio_token_1012audio_token_782audio_token_682audio_token_1012audio_token_743audio_token_589audio_token_155audio_token_195audio_token_589audio_token_195audio_token_195audio_token_480audio_token_873audio_token_821audio_token_195audio_token_705audio_token_179audio_token_155audio_token_660audio_token_69audio_token_564audio_token_685audio_token_8audio_token_589audio_token_69audio_token_685audio_token_589audio_token_155audio_token_69audio_token_301audio_token_69audio_token_100audio_token_37audio_token_195audio_token_301audio_token_589audio_token_810audio_token_750audio_token_416audio_token_589audio_token_564audio_token_69audio_token_589audio_token_566audio_token_810audio_token_317audi

In [ ]:
# Remove audio tokens such that length is even
if len(audio_tokens) % 2 != 0:
    audio_tokens = audio_tokens[:-1] 
len(audio_tokens)

1010

In [ ]:
def decode_audio_tokens_into_wav_file(audio_tokens, filename):
    number_of_codebooks = 2
    number_of_samples = len(tokens) // number_of_codebooks

    frame = torch.zeros(1, number_of_codebooks, number_of_samples, dtype=torch.long)

    for sample in range(number_of_samples):
        for codebook in range(number_of_codebooks):
            frame[0, codebook, sample] = audio_tokens[sample * number_of_codebooks + codebook]
    
    frames = [(frame, None)]

    with torch.no_grad():
        wav = encodec_model.decode(frames)

    torchaudio.save(filename, wav[0, :, :], encodec_model.sample_rate)

In [ ]:
# print largest audio token
print(max(audio_tokens))
print(min(audio_tokens))
print(audio_tokens)


1012
8
[195, 69, 195, 589, 743, 589, 195, 64, 195, 675, 682, 981, 155, 69, 458, 564, 64, 195, 155, 1012, 782, 682, 1012, 743, 589, 155, 195, 589, 195, 195, 480, 873, 821, 195, 705, 179, 155, 660, 69, 564, 685, 8, 589, 69, 685, 589, 155, 69, 301, 69, 100, 37, 195, 301, 589, 810, 750, 416, 589, 564, 69, 589, 566, 810, 317, 71, 71, 69, 782, 566, 301, 564, 924, 64, 64, 69, 133, 999, 743, 69, 564, 69, 923, 355, 750, 1012, 355, 564, 949, 660, 1012, 685, 663, 368, 694, 533, 301, 675, 589, 743, 589, 418, 675, 301, 564, 1012, 355, 409, 564, 355, 675, 660, 1004, 8, 778, 301, 355, 155, 705, 675, 301, 425, 873, 195, 1012, 778, 566, 195, 368, 564, 64, 195, 516, 750, 486, 968, 69, 533, 155, 564, 665, 377, 782, 750, 663, 155, 968, 525, 675, 564, 195, 155, 177, 195, 677, 64, 778, 155, 743, 750, 64, 155, 607, 743, 533, 750, 64, 195, 782, 516, 195, 155, 994, 195, 660, 782, 177, 743, 750, 195, 705, 533, 177, 158, 377, 782, 589, 368, 155, 525, 743, 533, 1012, 155, 564, 778, 377, 981, 486, 564, 155, 1012, 

In [ ]:
print(len(audio_tokens))

1010


In [ ]:

decode_audio_tokens_into_wav_file(audio_tokens, f"test.wav")

RuntimeError: Calculated padded input size per channel: (6). Kernel size: (7). Kernel size can't be greater than actual input size

In [ ]:
for i in range(len(audio_tokens), 0, -1):
    print(i)
    try:
        decode_audio_tokens_into_wav_file(audio_tokens[:i], f"test{i}.wav")
        print("success")
        break
    except:
        continue

1008
1007
1006
1005
1004
1003
1002
1001
1000
999
998
997
996
995
994
993
992
991
990
989
988
987
986
985
984
983
982
981
980
979
978
977
976
975
974
973
972
971
970
969
968
967
966
965
964
963
962
961
960
959
958
957
956
955
954
953
952
951
950
949
948
947
946
945
944
943
942
941
940
939
938
937
936
935
934
933
932
931
930
929
928
927
926
925
924
923
922
921
920
919
918
917
916
915
914
913
912
911
910
909
908
907
906
905
904
903
902
901
900
899
898
897
896
895
894
893
892
891
890
889
888
887
886
885
884
883
882
881
880
879
878
877
876
875
874
873
872
871
870
869
868
867
866
865
864
863
862
861
860
859
858
857
856
855
854
853
852
851
850
849
848
847
846
845
844
843
842
841
840
839
838
837
836
835
834
833
832
831
830
829
828
827
826
825
824
823
822
821
820
819
818
817
816
815
814
813
812
811
810
809
808
807
806
805
804
803
802
801
800
799
798
797
796
795
794
793
792
791
790
789
788
787
786
785
784
783
782
781
780
779
778
777
776
775
774
773
772
771
770
769
768
767
766
765
764
763
762
761